## Project 1: Predict the Housing Prices in Ames

#### [1. Introduction](#1)
##### [1.1. Input Data](#1.1)
#### [3. Results](#)

#### [x. References](#)
#### [x. Data Dictionary](#data_dict)

## <a id = '1'>1. Introduction<a/>

- The goal of this project is to predict the final price of a home (in log scale) using Ames housing dataset availalble on <a href="https://www.kaggle.com/c/house-prices-advanced-regression-techniques">Kaggle</a>. This is primarily a regression problem.

- Two models will be built:
    1. using linear regression with Lasso or Ridge or Elasticnet penalty
    2. using tree based models such as random forest or boosting tree.

### <a id='1.1'> 1.1 Input Data<a/>

- There are 83 columns and 2930 row objects. Explaination of each column can be found in the [Data Dictionary](#data_dict).
    

In [94]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)

In [95]:
ames_data = pd.read_csv('Ames_data.csv')
ames_data.head()

,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Longitude,Latitude,Sale_Price
0,526301100,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,No_Fence,None,0,5,2010,WD,Normal,-93.619754,42.054035,215000
1,526350040,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,...,Minimum_Privacy,None,0,6,2010,WD,Normal,-93.619756,42.053014,105000
2,526351010,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,No_Fence,Gar2,12500,6,2010,WD,Normal,-93.619387,42.052659,172000
3,526353030,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,...,No_Fence,None,0,4,2010,WD,Normal,-93.617320,42.051245,244000
4,527105010,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,...,Minimum_Privacy,None,0,3,2010,WD,Normal,-93.638933,42.060899,189900


In [96]:
len(ames_data)

2930

### <a id='1.2'> 1.2 Test Ids<a/>
    
- There are 10 different test id list, each containing 879 objects

In [43]:
dat_file = 'project1_testIDs.dat'
test_ids = np.loadtxt(dat_file).T
test_ids.shape

(10, 879)

### <a id='1.3'> 1.3 Evaluation Metric<a/>
    
- To measure the performance of the regression models, root mean squared error is used as a performance metric.

> $$ \sqrt{\frac{1}{\text{n.test}} \sum_{j=1}^{\text{n.test}} (\hat{y}_j - y_j)^2}$$

## <a id = '1'>2. Data pre-processing<a/>

Taking a look at the data structure of input data.

In [100]:
df_info = pd.DataFrame(columns=['column', 'Null Count', 'Data Type'])
for col in ames_data:
    Null_count  = sum(pd.isnull(ames_data[col]))
    dtype = ames_data[col].dtype
    df_info = df_info.append({'column': col, 'Null Count': Null_count, 'Data Type': dtype},
                            ignore_index = True)

In [102]:
df_info.head() # same as df.info()

,column,Null Count,Data Type
0,PID,0,int64
1,MS_SubClass,0,object
2,MS_Zoning,0,object
3,Lot_Frontage,0,int64
4,Lot_Area,0,int64


In [103]:
df_info[df_info['Data Type'] == 'int64']

,column,Null Count,Data Type
0,PID,0,int64
3,Lot_Frontage,0,int64
4,Lot_Area,0,int64
19,Year_Built,0,int64
20,Year_Remod_Add,0,int64
26,Mas_Vnr_Area,0,int64
34,BsmtFin_SF_1,0,int64
36,BsmtFin_SF_2,0,int64
37,Bsmt_Unf_SF,0,int64
38,Total_Bsmt_SF,0,int64


In [104]:
df_info[df_info['Data Type'] == 'object']

,column,Null Count,Data Type
1,MS_SubClass,0,object
2,MS_Zoning,0,object
5,Street,0,object
6,Alley,0,object
7,Lot_Shape,0,object
8,Land_Contour,0,object
9,Utilities,0,object
10,Lot_Config,0,object
11,Land_Slope,0,object
12,Neighborhood,0,object


In [105]:
df_info[df_info['Data Type'] == 'float64']

,column,Null Count,Data Type
59,Garage_Yr_Blt,159,float64
80,Longitude,0,float64
81,Latitude,0,float64


In [106]:
# ames_data.info()

## <a id = 'data_dict'>Data Dictionary<a/>
    
- Below is the data dictionary
    - PID: the Parcel identification number
    - SalePrice: the property's sale price in dollars. This is the target variable that you're trying to predict.
    - MSSubClass: The building class
    - MSZoning: The general zoning classification
    - LotFrontage: Linear feet of street connected to property
    -  LotArea: Lot size in square feet
    - Street: Type of road access
    -   Alley: Type of alley access
    -   LotShape: General shape of property
    - LandContour: Flatness of the property
    - Utilities: Type of utilities available
    -   LotConfig: Lot configuration
    - LandSlope: Slope of property
    - Neighborhood: Physical locations within Ames city limits
    -   Condition1: Proximity to main road or railroad
    -   Condition2: Proximity to main road or railroad (if a second is present)
    -  BldgType: Type of dwelling
    -   HouseStyle: Style of dwelling
    -  OverallCond: Overall condition rating
    -   YearBuilt: Original construction date
    -   YearRemodAdd: Remodel date
    -   RoofStyle: Type of roof
    -   RoofMatl: Roof material
    -   Exterior1st: Exterior covering on house
    -   MasVnrArea: Masonry veneer area in square feet
    -     ExterQual: Exterior material quality
    -    ExterCond: Present condition of the material on the exterior
    - BsmtQual: Height of the basement
    -    BsmtCond: General condition of the basement
    -   BsmtExposure: Walkout or garden level basement walls
    -    BsmtFinType1: Quality of basement finished area
    -    BsmtFinSF1: Type 1 finished square feet
    -    BsmtFinType2: Quality of second finished area (if present)
    -    BsmtFinSF2: Type 2 finished square feet
    -    BsmtUnfSF: Unfinished square feet of basement area
    -    TotalBsmtSF: Total square feet of basement area
    -   Heating: Type of heating
    -   HeatingQC: Heating quality and condition
    -   CentralAir: Central air conditioning
    -   Electrical: Electrical system
    -    1stFlrSF: First Floor square feet
    -    2ndFlrSF: Second floor square feet
    -    LowQualFinSF: Low quality finished square feet (all floors)
    -    GrLivArea: Above grade (ground) living area square feet
    -    BsmtFullBath: Basement full bathrooms
    -    BsmtHalfBath: Basement half bathrooms
    -    FullBath: Full bathrooms above grade
    -    HalfBath: Half baths above grade
    -    Bedroom: Number of bedrooms above basement level
    -    Kitchen: Number of kitchens
    -    KitchenQual: Kitchen quality
    -   TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
    -    Functional: Home functionality rating
    - FireplaceQu: Fireplace quality
    -  GarageType: Garage location
    - GarageYrBlt: Year garage was built
    -  GarageFinish: Interior finish of the garage
    -   GarageCars: Size of garage in car capacity
    -    GarageArea: Size of garage in square feet
    -   GarageQual: Garage quality
    -   GarageCond: Garage condition
    -   PavedDrive: Paved driveway
    -   WoodDeckSF: Wood deck area in square feet
    -   OpenPorchSF: Open porch area in square feet
    - EnclosedPorch: Enclosed porch area in square feet
    -   3SsnPorch: Three season porch area in square feet
    -   ScreenPorch: Screen porch area in square feet
    -   PoolArea: Pool area in square feet
    -   PoolQC: Pool quality
    -   Fence: Fence quality
    -   MiscFeature: Miscellaneous feature not covered in other categories
    -   MiscVal: $Value of miscellaneous feature
    -   MoSold: Month Sold
    -   YrSold: Year Sold
    -   SaleType: Type of sale
    -   SaleCondition: Condition of sale